## Convolutional Neural Network

- Each pixel between 0 and 255
- 2D array (image,gray scale)
- 3D array (r,g,b) (image,colorful)
- Input Image + Feature Detector = Feature Map
- Input Image > Convolutional Layer > Pooling Layer > flattening > ML
- Batch dimension, It allows for efficient processing of multiple examples simultaneously, leveraging parallelism.


#### Importing the libraries


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore
tf.Tensor

In [ ]:
tf.__version__

### Part 1 - Data Preprocessing


#### Preprocessing the Training set

- Feature Scale (rescale)
- Apply transformations
- Zoom
- Flip


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')


#### Preprocessing the Test set

- batch_size=32
- 8000 images in training_set => 8000/32 = 250
- 250 steps in each epoch


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

In [ ]:
import os

count = 0
for root_dir, cur_dir, files in os.walk(r'dataset/training_set'):
    count += len(files)
print('file count:', count)

### Part 2 - Building the CNN


#### Initialising the CNN


In [ ]:
#  Sequencial: input image > convlutioanl layer > input layer > hidden layer > output layer
#  Not Sequencial: graph layers
cnn = tf.keras.Sequential()

#### Step 1 - Convolution


In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3,
        activation='relu', input_shape=[64, 64, 3]))

#### Step 2 - Pooling


In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


#### Adding a second convolutional layer


In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#### Step 3 - Flattening


In [ ]:
cnn.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection


In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#### Step 5 - Output Layer


In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Part 3 - Training the CNN


#### Compiling the CNN


In [ ]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#### Training the CNN on the Training set and evaluating it on the Test set

**During each epoch:**

- Forward pass: The training data (training_set) is fed forward through the network, and predictions are made.
- Calculation of loss: The predictions are compared to the actual targets, and a loss value is computed to measure the network's performance.
- Backward pass (Backpropagation): The gradients of the loss with respect to the network's parameters (weights and biases) are calculated using backpropagation.
- Parameter update: The optimizer adjusts the network's parameters (weights and biases) based on the computed gradients to minimize the loss.


In [ ]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)
# time: 14m 58s


### Part 4 - Making a single prediction

- Neural networks, including Convolutional Neural Networks (CNNs), typically expect input data to be organized into batches, even if you're only predicting a single image.
- After np.expand_dims(test_image, axis=0), the shape(64,64,3) becomes (1, 64, 64, 3). The additional dimension at the beginning indicates that there is one image in the batch.


In [ ]:
import numpy as np
from keras.preprocessing import image  # type: ignore

test_image = image.load_img(
    'dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)  # 2DArray
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [ ]:
print(result)
print(result[0])
print(result[0][0])


In [ ]:
print(prediction)

#### Save The Model

- This will save the entire model architecture, including its weights, configuration, and optimizer state, to a single HDF5 file named "my_model.h5 or my_model.keras".


In [20]:
cnn.save('my_model.keras')